# 뉴스기사 전처리 후 저장하기

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

# 한글 폰트 사용을 위해서 세팅
from matplotlib import font_manager, rc
font_path = "C:/Windows/Fonts/malgun.ttf"
font = font_manager.FontProperties(fname=font_path).get_name()
rc('font', family=font)

In [2]:
import re
from pykospacing import Spacing
from konlpy.tag import Hannanum,Kkma,Komoran,Okt

In [3]:
import FinanceDataReader as fdr

In [4]:
# 띄어쓰기
def spacing_text(docs):
    spacing = Spacing()
    docs = spacing(docs)
    return docs

In [5]:
# 명사 추출
def han_noun(docs):
    han = Hannanum()
    docs = han.nouns(docs)
    return docs

In [6]:
# 모든 날짜 채워주기 (2016-01-01 ~ 2022-03-31)
def fill_date(df):
    
    # 날짜로 묶기
    df = df.groupby('날짜')['뉴스제목'].sum().to_frame()
    
    # 모든 날짜 채우기
    idx = pd.date_range('2016-01-01','2022-03-31')
    df.index = pd.DatetimeIndex(df.index)
    df = df.reindex(idx, fill_value= '')
    
    return df

In [7]:
# 한글,영어만 남기기
def clean_text(docs):
    docs = re.sub('[^가-힣A-Za-z ]', '', docs)
    docs = re.sub('\s+', ' ', docs)
    docs = '' if docs== ' ' else docs
    return docs

In [8]:
# 불용어 제거
stw_list = pd.read_csv('./data/stopwords-ko.txt')
def remove_stwords(docs):
    docs = [w for w in docs if not w in stw_list]
    docs = '' if docs== ' ' else docs
    docs = [w for w in docs if len(w)>1]
    return docs

In [25]:
df = pd.read_csv('./data/개별기업뉴스_전처리/AJ네트웍스.csv',index_col=0)
print(len(df))
df.head()

907


,날짜,뉴스제목,뉴스요약
1313,2016-01-04,"AJ네트웍스, SKB 등에업고 '고리사채급 위약금 폭탄'",AJ네트웍스가 SK브로드밴드를 통해 진행중인 '스마트렌탈'의 위약금 규정 및 취급상...
1312,2016-01-05,"AJ네트웍스, 스마트폰, 태블릿PC 렌탈사업 시작","국내 IT기기 렌탈의 선두주자인 AJ네트웍스에서 Mobile Device(스마트폰,..."
1311,2016-01-06,"AJ네트웍스, 스마트폰, 태블릿PC 등 Mobile Device 렌탈 사업 시작","국내 IT기기 렌탈의 선두주자인 AJ네트웍스에서 Mobile Device(스마트폰,..."
1332,2016-01-13,"AJ네트웍스, 파렛트및 고소장비 렌탈 신규 서비스 브랜드 런칭",AJ네트웍스(사장 반채운)가 파렛트와 고소장비 렌탈 부문의 신규 서비스 브랜드를 런...
1335,2016-01-22,대상·AJ네트웍스 등 비우량채도 잇달아 발행 성공,신용등급이 BBB+인 AJ네트웍스도 회사채 발행을 위해 지난 20일 투자자들을 대상...


In [27]:
df = fill_date(df)
df.head()

,뉴스제목
2016-01-01,
2016-01-02,
2016-01-03,
2016-01-04,"AJ네트웍스, SKB 등에업고 '고리사채급 위약금 폭탄'"
2016-01-05,"AJ네트웍스, 스마트폰, 태블릿PC 렌탈사업 시작"


In [4]:
df_n = df.groupby('날짜')['뉴스제목'].sum().to_frame()

In [11]:
df_n['뉴스'] = df_n['뉴스제목'].apply(clean_text)
df_n.head()

,뉴스제목,뉴스
날짜,,
2016-01-04,"AJ네트웍스, SKB 등에업고 '고리사채급 위약금 폭탄'",AJ네트웍스 SKB 등에업고 고리사채급 위약금 폭탄
2016-01-05,"AJ네트웍스, 스마트폰, 태블릿PC 렌탈사업 시작",AJ네트웍스 스마트폰 태블릿PC 렌탈사업 시작
2016-01-06,"AJ네트웍스, 스마트폰, 태블릿PC 등 Mobile Device 렌탈 사업 시작",AJ네트웍스 스마트폰 태블릿PC 등 Mobile Device 렌탈 사업 시작
2016-01-13,"AJ네트웍스, 파렛트및 고소장비 렌탈 신규 서비스 브랜드 런칭",AJ네트웍스 파렛트및 고소장비 렌탈 신규 서비스 브랜드 런칭
2016-01-22,대상·AJ네트웍스 등 비우량채도 잇달아 발행 성공,대상AJ네트웍스 등 비우량채도 잇달아 발행 성공


In [13]:
df_n['뉴스'] = df_n['뉴스'].apply(spacing_text)
df_n['뉴스'].head()

날짜
2016-01-04                 AJ네트웍스 SKB 등에 업고 고리 사채급 위약금 폭탄
2016-01-05                     AJ네트웍스 스마트폰 태블릿PC 렌탈 사업 시작
2016-01-06    AJ네트웍스 스마트폰 태블릿PC 등 Mobile Dev ice 렌탈 사업 시작
2016-01-13             AJ네트웍스 파렛트 및 고소장비 렌탈 신규 서비스 브랜드 런칭
2016-01-22                    대상 AJ네트웍스 등 비우량채도 잇달아 발행 성공
Name: 뉴스, dtype: object

In [19]:
df_n['뉴스'] = df_n['뉴스'].apply(han_noun)
df_n.head()

,뉴스제목,뉴스
날짜,,
2016-01-04,"AJ네트웍스, SKB 등에업고 '고리사채급 위약금 폭탄'","[AJ네트웍스, 등, 사채급, 위약금, 폭탄]"
2016-01-05,"AJ네트웍스, 스마트폰, 태블릿PC 렌탈사업 시작","[AJ네트웍스, 스마트폰, 태블릿PC, 렌탈, 사업, 시작]"
2016-01-06,"AJ네트웍스, 스마트폰, 태블릿PC 등 Mobile Device 렌탈 사업 시작","[AJ네트웍스, 스마트폰, 태블릿PC, 등, 렌탈, 사업, 시작]"
2016-01-13,"AJ네트웍스, 파렛트및 고소장비 렌탈 신규 서비스 브랜드 런칭","[AJ네트웍스, 파렛트, 고소장비, 렌탈, 신규, 서비스, 브랜드, 런칭]"
2016-01-22,대상·AJ네트웍스 등 비우량채도 잇달아 발행 성공,"[대상, AJ네트웍스, 등, 우량채, 발행, 성공]"


In [21]:
df_n['뉴스'] = df_n['뉴스'].apply(remove_stwords)
df_n.head()

,뉴스제목,뉴스
날짜,,
2016-01-04,"AJ네트웍스, SKB 등에업고 '고리사채급 위약금 폭탄'","[AJ네트웍스, 사채급, 위약금, 폭탄]"
2016-01-05,"AJ네트웍스, 스마트폰, 태블릿PC 렌탈사업 시작","[AJ네트웍스, 스마트폰, 태블릿PC, 렌탈, 사업, 시작]"
2016-01-06,"AJ네트웍스, 스마트폰, 태블릿PC 등 Mobile Device 렌탈 사업 시작","[AJ네트웍스, 스마트폰, 태블릿PC, 렌탈, 사업, 시작]"
2016-01-13,"AJ네트웍스, 파렛트및 고소장비 렌탈 신규 서비스 브랜드 런칭","[AJ네트웍스, 파렛트, 고소장비, 렌탈, 신규, 서비스, 브랜드, 런칭]"
2016-01-22,대상·AJ네트웍스 등 비우량채도 잇달아 발행 성공,"[대상, AJ네트웍스, 우량채, 발행, 성공]"


In [22]:
bgn_date = '2016-01-01'
end_date = '2016-02-01'

token_lst = df_n['뉴스'].loc[bgn_date:end_date]
token_lst

날짜
2016-01-04                       [AJ네트웍스, 사채급, 위약금, 폭탄]
2016-01-05            [AJ네트웍스, 스마트폰, 태블릿PC, 렌탈, 사업, 시작]
2016-01-06            [AJ네트웍스, 스마트폰, 태블릿PC, 렌탈, 사업, 시작]
2016-01-13    [AJ네트웍스, 파렛트, 고소장비, 렌탈, 신규, 서비스, 브랜드, 런칭]
2016-01-22                    [대상, AJ네트웍스, 우량채, 발행, 성공]
Name: 뉴스, dtype: object

In [24]:
kw_dict = dict()
for tokens in token_lst:

    for word in tokens:
        if not word in kw_dict.keys():
            kw_dict[word] = 1
        else:
            kw_dict[word] += 1

kw_dict = dict(sorted(kw_dict.items(), key = lambda x: x[1]))

kw_dict

{'사채급': 1,
 '위약금': 1,
 '폭탄': 1,
 '파렛트': 1,
 '고소장비': 1,
 '신규': 1,
 '서비스': 1,
 '브랜드': 1,
 '런칭': 1,
 '대상': 1,
 '우량채': 1,
 '발행': 1,
 '성공': 1,
 '스마트폰': 2,
 '태블릿PC': 2,
 '사업': 2,
 '시작': 2,
 '렌탈': 3,
 'AJ네트웍스': 5}

# 합친코드

In [37]:
df = pd.read_csv('./data/개별기업뉴스_전처리/AJ네트웍스.csv',index_col=0)
df = fill_date(df)

In [38]:
df.head()

,뉴스제목
2016-01-01,
2016-01-02,
2016-01-03,
2016-01-04,"AJ네트웍스, SKB 등에업고 '고리사채급 위약금 폭탄'"
2016-01-05,"AJ네트웍스, 스마트폰, 태블릿PC 렌탈사업 시작"


In [31]:
path = './data/개별기업뉴스_전처리/'
file_list = os.listdir(path)
for file in file_list[:3]:
    corp_name = file[:-4]
    print(corp_name)

AJ네트웍스
AK홀딩스
BGF


In [53]:
db_list = pd.read_csv('./data/개별기업명_511',index_col=0)
db_list.head()

,stock_code,회사이름
0,40,KR모터스
1,50,경방
2,80,하이트진로
3,100,유한양행
4,120,CJ대한통운


In [32]:
# 완성코드
db_list = pd.read_csv('./data/개별기업명_511.csv',index_col=0)
path = './data/개별기업뉴스_전처리/'
file_list = os.listdir(path)

for file in file_list[200:300]:
    corp_name = file[:-4]
    print(corp_name)
    if corp_name in db_list['회사이름'].values:
        file_path = os.path.join(path,file)
        df = pd.read_csv(file_path,index_col=0)
        df = fill_date(df)
        df['뉴스'] = df['뉴스제목'].apply(clean_text)
        df['뉴스'] = df['뉴스'].apply(han_noun)
        df['뉴스'] = df['뉴스'].apply(remove_stwords)
        df.to_csv('./data/개별기업뉴스_자연어처리/{}.csv'.format(corp_name))

대우부품
대우조선해양


KeyboardInterrupt: 

In [ ]:
# 0:90
# 90:200

In [19]:
df = pd.read_csv('./data/개별기업뉴스_전처리/LG전자.csv',index_col=0)
df = fill_date(df)
df = df.iloc[:20]

In [20]:
df['뉴스'] = df['뉴스제목'].apply(clean_text)
df['뉴스'] = df['뉴스'].apply(han_noun)
df['뉴스'] = df['뉴스'].apply(remove_stwords)

In [21]:
df

,뉴스제목,뉴스
2016-01-01,,[]
2016-01-02,,[]
2016-01-03,"LG전자, 더 선명해진 ‘슈퍼 울트라HD TV’ 공개","[전자, 선명, 슈퍼, 울트라HD, 공개]"
2016-01-04,"LG전자 조성진 사장, “2016년은 ‘주도적 변화 창출’의 해”","[전자, 조성진, 사장, 주도적, 변화, 창출]"
2016-01-05,"LG전자, 보급형 스마트폰 ‘K 시리즈’ ‘CES 2016’서 공개LG전자, CES...","[전자, 보급형, 스마트폰, 시리즈, 공개LG전자, 글로벌, 시장선, 전략제품]"
2016-01-06,LG전자 안승권 사장 “LG 시그니처에 기술과 디자인의 정수 담았다”,"[전자, 안승권, 사장, 시그니처, 기술, 디자인, 정수]"
2016-01-07,,[]
2016-01-08,"LG전자, 超프리미엄 빌트인 브랜드 ‘시그니처 키친 스위트’ 론칭","[전자, 프리미엄, 빌트, 브랜드, 시그니처, 키친, 스위트, 론칭]"
2016-01-09,,[]
2016-01-10,"LG전자, 프리미엄급 디자인 보급형 스마트폰 ‘K10’ 국내 출시","[전자, 프리미엄급, 디자인, 보급형, 스마트폰, 국내, 출시]"


In [45]:
db_list = pd.read_csv('./data/DB_list_stock_code.csv',index_col=0)
db_list['stock_code'] = db_list['stock_code'].apply(lambda x: str(x).zfill(6))
db_list

,stock_code
0,000040
1,000050
2,000080
3,000100
4,000120
...,...
506,214330
507,214420
508,226320
509,227840


In [50]:
len(corp_list)

511